# Notebook test 220129

In [1]:
fileName = "part1_Mesh.off"
## MUST put model in 'data' folder

In [2]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

import os
root_folder = os.getcwd()

In [3]:
## Load a mesh in OFF format
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", fileName))

## Print the vertices and faces matrices 
print("Vertices: ", len(v))
print("Faces: ", len(f))

Vertices:  76027
Faces:  151765


tutorial = https://libigl.github.io/libigl-python-bindings/tut-chapter1/

# Gaussian Curvature (tutorial)

In [4]:
k = igl.gaussian_curvature(v, f)
plot(v, f, k)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -30.…

"compute the massmatrix and divide the gaussian curvature values by area to get the integral average"

In [5]:
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
minv = sp.sparse.diags(1 / m.diagonal())
kn = minv.dot(k)
plot(v, f, kn)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -30.…

# Curvature directions (tutorial)

In [6]:
l = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

minv = sp.sparse.diags(1 / m.diagonal())

hn = -minv.dot(l.dot(v))
h = np.linalg.norm(hn, axis=1)
plot(v, f, h)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -30.…

"determining principal curvatures is via quadric fitting"

In [7]:
v1, v2, k1, k2 = igl.principal_curvature(v, f)
h2 = 0.5 * (k1 + k2)
p = plot(v, f, h2, shading={"wireframe": False}, return_plot=True)

avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -30.…

2

# Laplacian (Tutorial)

In [8]:
from scipy.sparse.linalg import spsolve

v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", fileName))
l = igl.cotmatrix(v, f)

n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)


vs = [v]
cs = [c]
for i in range(10):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    s = (m - 0.001 * l)
    b = m.dot(v)
    v = spsolve(s, m.dot(v))
    n = igl.per_vertex_normals(v, f)*0.5+0.5
    c = np.linalg.norm(n, axis=1)
    vs.append(v)
    cs.append(c)


p = subplot(vs[0], f, c, shading={"wireframe": False}, s=[1, 4, 0])
subplot(vs[3], f, c, shading={"wireframe": False}, s=[1, 4, 1], data=p)
subplot(vs[6], f, c, shading={"wireframe": False}, s=[1, 4, 2], data=p)
subplot(vs[9], f, c, shading={"wireframe": False}, s=[1, 4, 3], data=p)